[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/agents-sdk-course/blob/main/chapters/03-tools.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/agents-sdk-course/blob/main/chapters/03-tools.ipynb)

#### [Agents SDK Course](https://www.aurelio.ai/course/agents-sdk)

# Tools

Agents SDK provides various approaches for tool-use, including pre-built tools and all features we need to develop custom tools. In this chapter we'll learn _everything_ there is about tools and tool-use.

## Setup

We'll first need to install `openai-agents` like so:

In [ ]:
!pip install -qU \
    "openai-agents==0.1.0"

To follow along you will need your [OpenAI API key](https://platform.openai.com/api-keys).

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
    getpass.getpass("OpenAI API Key: ")

### Pre-Packaged Tools

The Agents SDK team provides us with a few pre-packaged tools that can be used to help with common tasks, these include:
- The `WebSearchTool` lets an agent search the web.
- The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.
- The `ComputerTool` allows automating computer use tasks.

We can start by creating an agent that uses the `WebSearchTool` to search the web. To do this we need to define an agent with the `tools` parameter set to a list containing the `WebSearchTool` object.

In [2]:
from agents import Agent, WebSearchTool

agent = Agent(
    name="Web Search Agent",
    instructions=(
        "You are a web search agent that searches the web for information to answer the user's "
        "queries with high accuracy and precision."
    ),
    tools=[
        WebSearchTool()
    ],
    model='gpt-4.1-mini'
)

We can look at the tools that are currently available to our agent via the `tools` attribute connected to our agent object.

In [3]:
agent.tools

[WebSearchTool(user_location=None, search_context_size='medium')]

Now we can use the `Runner` object to run the agent, asking specifically for the up-to-date information on weather or news related topics that require a web search.

In [4]:
from agents import Runner 

query = "What are the current headlines in the news?"

result = await Runner.run(
    starting_agent=agent,
    input=query
)

[non-fatal] Tracing: server error 500, retrying.
[non-fatal] Tracing: server error 500, retrying.


As we can see, the agent can successfully search the web and return the results, however, if the response can sometimes be out-of-date, due to the LLM thinking that the current date is earlier than it actually is.

In [5]:
print(result.final_output)

Here are some of the latest news headlines as of July 10, 2025:

**International Affairs**

- **U.S. Sanctions North Korean Official**: The U.S. has imposed sanctions on Song Kum Hyok, a member of Kim Jong Un's spy agency, accusing him of orchestrating a scheme where North Korean cyber operatives posed as U.S. remote IT workers to infiltrate companies. ([cbsnews.com](https://www.cbsnews.com/?utm_source=openai))

- **Trump and Netanyahu Discuss Gaza Conflict**: President Trump and Israeli Prime Minister Benjamin Netanyahu are engaged in talks aimed at ending the ongoing conflict in Gaza, with a potential ceasefire on the table. ([cbsnews.com](https://www.cbsnews.com/?utm_source=openai))

**U.S. News**

- **Texas Floods Claim Over 100 Lives**: Flash floods in central Texas, particularly in Kerr County, have resulted in the deaths of more than 100 individuals, including young girls and camp staff. ([cbsnews.com](https://www.cbsnews.com/?utm_source=openai))

- **Milan Airport Incident**: M

### Custom Tools

Now although the pre-packaged tools are useful, for specific tasks we need specific tools tailored to the problem we are trying to solve. To do this, we need to create our own custom tools.

We can start by creating a tool that fetches the current time.

First we need to import the `function_tool` decorator from the `agents` module.

Then we need to define our function, this needs to be an async function for the agent to work.

Then using the `function_tool` decorator, we can create a tool from our function.

In [13]:
from agents import function_tool
from datetime import datetime

# we can override the name of a tool using the name_override parameter
@function_tool(name_override="fetch_current_time")
async def fetch_time() -> str:
    """Fetch the current time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can check if the `fetch_time` object is a `FunctionTool` object by using the `isinstance` function.

In [14]:
from agents import FunctionTool

if isinstance(fetch_time, FunctionTool):
    print("This is a FunctionTool object")

This is a FunctionTool object


As this is now a `FunctionTool` object, we can print out specific properties of the tool.

The `name` property might be slightly different to the name we defined in the function, this is because the `function_tool` decorator has a `name_override` parameter that we can use to override the name of the tool.

The `description` attribute is the first line of the docstring of the function.

The `params_json_schema` attribute is the JSON schema for the parameters of the tool.

In [15]:
import json

print("Name:", fetch_time.name)
print("Description:", fetch_time.description)
print("Params:", json.dumps(fetch_time.params_json_schema, indent=2))

Name: fetch_current_time
Description: Fetch the current time.
Params: {
  "properties": {},
  "title": "fetch_current_time_args",
  "type": "object",
  "additionalProperties": false,
  "required": []
}


We want to test our agent, but first we need to define an output class that the agent will use to return its response. This way we can ensure the response is correct as well as the approach taken to get the response.

For this, we will use the `BaseModel` class from `pydantic` and the `Field` class to define the response and approach taken.

In [ ]:
from pydantic import BaseModel, Field

class OutputClass(BaseModel):
    response: str = Field(
        description="The agent's response to the user's query"
    )
    approach_taken: str = Field(
        description="The method or strategy the agent used to arrive at the response"
    )

Now we can define our agent, with the only difference being the `output_type` parameter set to our `OutputClass`.

In [20]:
agent = Agent(
    name="Time Agent",
    instructions="You are a time agent that fetches the current time.",
    tools=[fetch_time],
    model='gpt-4.1-mini',
    output_type=OutputClass
)

Now as before we will use the `Runner` object to run the agent.

In [21]:
query = "What is the current time?"

result = await Runner.run(
    starting_agent=agent,
    input=query
)

In [23]:
result.final_output.response, result.final_output.approach_taken

('The current time is 18:57 on July 10, 2025.',
 "I fetched the current time using the system's time query function to provide you with the exact and up-to-date time.")

This time we have a slightly different output due to our `OutputClass` definition. Instead of a simple string, we have a dictionary with two outputs, the `response` and the `approach_taken`.

If the agent worked as expected, we should see the correct time, and the approach should mention the use of the `fetch_time` tool.

In [24]:
print("Response:", result.final_output.response)
print("Approach Taken:", result.final_output.approach_taken)

Response: The current time is 18:57 on July 10, 2025.
Approach Taken: I fetched the current time using the system's time query function to provide you with the exact and up-to-date time.


We have just defined a tool using the `function_tool` decorator, but we can also define tools using the `FunctionTool` object.

When defining the tool this way, we need to ensure that the `params_json_schema` attribute has a `additionalProperties` key set to `False`. By default this will not be included in the schema, so we add a `Config` class to the `FunctionTool` object with the `extra` parameter set to `"forbid"` to force the schema to include it.

In [36]:
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool

class FunctionArgs(BaseModel):
    x: float = Field(description="The first number to multiply")
    y: float = Field(description="The second number to multiply")

    class Config:
        extra = "forbid"  # This adds additionalProperties: False to the schema

async def multiply_numbers(ctx: RunContextWrapper[Any], args: str) -> float:
    parsed = FunctionArgs.model_validate_json(args)
    return parsed.x * parsed.y

multiply_tool = FunctionTool(
    name="multiply_numbers", # name of the tool
    description="Multiplies two numbers", # description of the tool
    params_json_schema=FunctionArgs.model_json_schema(), # schema of the tool
    on_invoke_tool=multiply_numbers, # function to call when the tool is invoked
)

Now we can check the properties of the tool to ensure it is defined correctly. 

Note, if you do not include the `Config` class, the `"additionalProperties": false` will not be included in the schema, and when the tool is invoked this will cause an error...

In [37]:
print("Name:", multiply_tool.name)
print("Description:", multiply_tool.description)
print("Params:", json.dumps(multiply_tool.params_json_schema, indent=2))

Name: multiply_numbers
Description: Multiplies two numbers
Params: {
  "additionalProperties": false,
  "properties": {
    "x": {
      "description": "The first number to multiply",
      "title": "X",
      "type": "number"
    },
    "y": {
      "description": "The second number to multiply",
      "title": "Y",
      "type": "number"
    }
  },
  "required": [
    "x",
    "y"
  ],
  "title": "FunctionArgs",
  "type": "object"
}


Now we are all set to define our agent and use the tool.

This time we will use the same `OutputClass` as before to ensure the response is correct including the approach taken.

In [38]:
agent = Agent(
    name="Multiply Agent",
    instructions="You are a multiply agent that multiplies two numbers always by using the tool.",
    tools=[multiply_tool],
    model='gpt-4.1-mini',
    output_type=OutputClass
)

As before we can define our query before using the `Runner` object to run the agent.

In [39]:
query = "multiply 3.41 by 7.2"

result = await Runner.run(
    starting_agent=agent,
    input=query
)

In [40]:
result.final_output

OutputClass(response='The product of 3.41 and 7.2 is 24.552.', approach_taken='Used the multiplication tool to multiply the two given numbers 3.41 and 7.2.')

Next we can print our final outputs to ensure the response is correct.

In [41]:
print("Response:", result.final_output.response)
print("Approach Taken:", result.final_output.approach_taken)

Response: The product of 3.41 and 7.2 is 24.552.
Approach Taken: Used the multiplication tool to multiply the two given numbers 3.41 and 7.2.


### Agents As Tools

The last thing we will look at is how to use agents as tools, as the Agents SDK strongly encourages the use of handoffs and agents as tools to build more complex systems.

First we want to define our bottom level agents (in this case the "tools" we will use). For this example we will redefine the `time_agent` and `multiply_agent` from before.

In [42]:
multiply_agent = Agent(
    name="Multiply Agent",
    instructions="""You are a multiply agent that multiplies two
    numbers always by using the tool. Make sure when returning your
    response you include the agent that provided the information 
    along with any additional tool calls used within the agent.""",
    tools=[multiply_tool],
    model='gpt-4.1-mini',
)

time_agent = Agent(
    name="Time Agent",
    instructions="""You are a time agent that fetches the current
    time. Make sure when returning your response you include the
    agent that provided the information along with any additional
    tool calls used within the agent.""",
    tools=[fetch_time],
    model='gpt-4.1-mini',
)

Next we can define our top level agent, the orchestrator agent. This agent will use the `multiply_agent` and `time_agent` as tools. 

Within the `as_tool` method, we can set the `tool_name` and `tool_description` parameters to the name and description of the tool for additional clarity.

In [43]:
orchestrator_agent = Agent(
    name="Orchestrator Agent",
    instructions="""You are an orchestrator agent that uses the
    tools given to you to complete the user's query.""",
    tools=[
        multiply_agent.as_tool(
            tool_name="multiply_numbers_agent",
            tool_description="Multiply two numbers",
        ),
        time_agent.as_tool(
            tool_name="fetch_current_time_agent",
            tool_description="Fetch the current time",
        ),
    ],
    model='gpt-4.1-mini',
    output_type=OutputClass
)

Next we can define our query before using the `Runner` object to run the agent.

In [44]:
query = "what time is it?"

result = await Runner.run(
    starting_agent=orchestrator_agent,
    input=query
)

Then finally we can print the response and approach taken by the agent.

In [45]:
print("Response:", result.final_output.response)
print("Approach Taken:", result.final_output.approach_taken)

Response: The current time is 19:11:56 on July 10, 2025.
Approach Taken: I used a built-in function to fetch the current time and date.


---